In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
import ast
import pickle

warnings.filterwarnings('ignore')
tqdm.pandas()

import requests
import base64
import json

# Azure GPT API 설정
GPT4V_ENDPOINT = "https://apeus-cst-openai.openai.azure.com/openai/deployments/gpt4o/chat/completions?api-version=2024-02-15-preview"
GPT4V_KEY = "fb73c5cdb51d4167b9eb41345d059bfe"

# 헤더 설정
headers = {
    "Content-Type": "application/json;",
    "api-key": GPT4V_KEY,
}

def respond(payload):
    response = requests.post(GPT4V_ENDPOINT, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
        
    else:
        return f"Error {response.status_code}: {response.text}"

In [3]:
internal_df = pd.read_csv("internal_characteristics.csv", encoding = "utf-8-sig", index_col = 0)
internal_df["focus_item"] = internal_df["focus_item"].apply(lambda x : ast.literal_eval(x))

external_df = pd.read_csv("external_characteristics.csv", encoding = "utf-8-sig")
# external_df["internal_item"] = external_df["internal_item"].apply(lambda x : ast.literal_eval(x))
external_df["external_item"] = external_df["external_item"].apply(lambda x : ast.literal_eval(x))
idf = dict(zip(internal_df["internal_cluster"], internal_df["characteristics"]))
external_df["internal_characteristics"] = [idf[x] for x in external_df["internal_cluster_num"]]

In [4]:
# 내부 데이터로 피부 고민 찾기
internal_gomin = []

system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
    
user1 = """
나는 너에게 고객의 특징을 알려줄거야. 하지만 지금 정의된 특징들이 너무 고객에게 이야기하기에는 너무 직접적인 내용을 담고 있어.
그 특징을 가진 고객이 어떤 피부 고민을 가지고 있을지, 또는 어떤 화장품 제품을 원할지, 아래 조건들을 참고해서 말해줘.
조건 1. 고객의 특징이 언급하고 있는 정보들을 반드시 모두 사용해야만 해.
조건 2. 환경 보호, 자연 보호, 클린 뷰티는 절대로 포함되어서는 안 돼.
조건 3. 주어진 정보들을 직접적으로 사용하기보다, 정보들을 유기적으로 조합해서 감성적인 특징을 제시해줬으면 좋겠어.
"""
assistant1 = "알겠습니다. 고객의 특징을 알려주세요."

user2 = "한 군데에만 있으면 좀이 쑤셔 여기저기 돌아다녀야 직성이 풀리고, 여행 시 들고다닐 간편한 아이템을 원하는 고객."
assistant2 = "휴대성과 사용성이 간편한 제품을 원함. 잦은 환경변화로 인해 피부 트러블 올라 오는 경우가 많아 늘 사용하던 제품이나 저자극의 순한 제품을 원함."

user3 = "에이블리/지그재그에서 패션 쇼핑하고 올리브영을 자주가며 OTT를 구독하고 문화생활을 많이 하는 고객. 카드사, 온라인 할인 혜택을 잘 사용하는 고객."
assistant3 = "피지 분비가 많은 시기, 호르몬 변화에 의한 피부 트러블이 자주 올라옴. 여드름 흉터, 블랙헤드 등 트러블 자국과 고르지 않은 피부결에 대해 고민이 높음."

user4 = "뷰티관심러로서 뷰티 관리를 철저히 하는 고객. 피부과, 피부 에스테틱을 자주 다니며 헤어/네일 등 미용 관리 역시 높은 관심을 가지고 있는 고객. 심지어 이너뷰티까지 중요성까지 알고 있기에 샐러드 등 건강식단을 중요시하며, 실내운동도 꾸준히 하는 고객."
assistant4 = "일상 생활 중 자극을 받아 생겨난 트러블, 붉은 기, 전반적인 피부 장벽 및 컨디션 저하를 해결하고 싶어함. 메이크업이 잘 먹는 최적의 상태의 피부(촉촉하고 수분 가득하고, 피부결이 고르게 정돈된 피부)를 원함"

for i in tqdm(internal_df["characteristics"]):

    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": user2},
            {"role": "assistant", "content": assistant2},
            {"role": "user", "content": user3},
            {"role": "assistant", "content": assistant3},
            {"role": "user", "content": user4},
            {"role": "assistant", "content": assistant4},
            {"role": "user", "content": i}]}
    
    decision = True
    while decision:
        try:
            r = respond(payload)
            if "죄송" in r:
                pass
            else:
                internal_gomin.append(r)
                decision = False
        except:
            pass

internal_df["concern"] = internal_gomin

100%|██████████| 32/32 [01:32<00:00,  2.88s/it]


In [5]:
# 외부 데이터로 피부 고민 찾기
external_gomin = []

for a, i in tqdm(enumerate(external_df["characteristics"])):

    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": user2},
            {"role": "assistant", "content": assistant2},
            {"role": "user", "content": user3},
            {"role": "assistant", "content": assistant3},
            {"role": "user", "content": i + " " + external_df["internal_characteristics"].loc[a]}]}
    
    decision = True
    while decision:
        try:
            r = respond(payload)
            if "죄송" in r:
                pass
            else:
                external_gomin.append(r)
                decision = False
        except:
            pass

external_df["concern"] = external_gomin

97it [06:30,  4.03s/it]


In [8]:
# df = pd.read_csv("item2.csv")

# target_items = {"그린티씨드세럼" : 111705279, "그린티씨드크림" : 111770419,
#                 "레티놀시카앰플" : 131173569, "레티놀PDRN앰플" : 131175032,
#                 "비타C엔자임세럼" : 131173960}

# result = {}

# for i in tqdm(target_items):
#     temp = list(df[df['RPRS_PRD_CD'] == target_items[i]]["GOODCNT"])

#     system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
#     user1 = f"""
#     나는 너에게 {i}에 대한 고객 리뷰들을 제공해 줄거야. 그 리뷰들을 요약해서, {i}가 어떤 제품인지 자세하게 말해줘.
#     """
#     assistant1 = "알겠습니다. 리뷰를 제공해 주세요."

#     payload = {
#         "messages": [
#             {"role": "system", "content": system1},
#             {"role": "user", "content": user1},
#             {"role": "assistant", "content": assistant1},
#             {"role": "user", "content": str(temp)}]}

#     decision = True
#     while decision:
#         try:
#             result[i] = respond(payload)
#             decision = False
#         except:
#             pass
    
# with open(file='recommended_contents.pickle', mode='wb') as f:
#     pickle.dump(result, f)

In [9]:
cp = {
    "그린티씨드세럼" : "대한민국 판매 1위 수분에센스. 단숨에 속건조 해결",
    "그린티씨드크림" : "외부자극으로 성난 피부 수분 진정. 재구매율 1위. 써 본 사람은 다 아는 확실한 수분 진정 효과. 수분을 빈틈 없이 꽉 채워 화장이 잘 먹는 수분크림",
    "레티놀시카앰플" : "국민 트러블 앰플. 갓 생긴 트러블부터 모공, 잡티, 흔적까지 트러블 끝장 케어. 반복되는 트러블 고리의 악순환을 끊는 #레티놀시카. 트러블, 모공, 잡티, 흔적 고민 지우고 깨끗하고 매끈매끈한 깐달걀 피부결",
    "레티놀PDRN앰플" : "모공결부터 속광까지 #바르는 스킨 부스터. 매끈한 모공결 & 촘촘한 피부광의 스킨부스터 앰플. 주름, 모공, 탄력 복잡한 피부 고민 이거 하나로 끝! 당신의 피부를 전방위적으로 개선합니다."
}

In [10]:
# 내부 데이터 타겟팅

internal_result = []
for i in tqdm(internal_df["concern"]):
    
    system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
    user1 = f"""
    나는 너에게 특정 고객군의 피부 고민을 알려줄거야.
    그 특징을 바탕으로, [그린티씨드세럼, 그린티씨드크림, 레티놀시카앰플, 레티놀PDRN앰플] 중에서 가장 적절한 제품 하나를 추천해줘.
    그리고 그 추천이 얼마나 자신있는지 0% - 100%로 표시해준 후, 추천을 한 이유도 함께 설명해줘.
    우리가 선호하는 답변의 포맷은 다음과 같아: '레티놀시카앰플; 75%; 이 고객군은 피부가 예민하기 때문에, 순한 제품이 어울릴 것으로 예상됩니다.'
    각 제품에 대한 설명은 'Items'를 참조해.
    Items: {cp}
    """
    assistant1 = "알겠습니다. 고객군의 특징을 제공해 주세요."

    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": i}]}

    decision = True
    while decision:
        try:
            internal_result.append(respond(payload))
            decision = False
        except:
            pass

one = []
two = []
three = []
for i in internal_result:
    temp = i.split(";")
    one.append(temp[0])
    two.append(temp[1])
    three.append(temp[2])

internal_df["recommended_item"] = one
internal_df["confidence"] = two
internal_df["reason_of_recommendation"] = three
internal_df

100%|██████████| 32/32 [01:29<00:00,  2.79s/it]


,internal_cluster,focus_item,characteristics,concern,recommended_item,confidence,reason_of_recommendation,message0,message1,message2,message3,message4
0,0-0,"[131170000, 131171318, 131173347]","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...","자연스럽고 매혹적인 눈매와, 매끈하고 깨끗한 피부를 원함. 여름철 습하고 더운 날씨...",레티놀PDRN앰플,80%,"이 고객군은 매끈하고 깨끗한 피부를 원하며, 여름철에도 지속력 있는 메이크업을 추...",액티브 라이프스타일을 즐기지만 피부 톤도 신경 쓰세요? 비타C엔자임세럼으로 생기 넘...,"활동적인 라이프스타일을 즐기신다면, 피부 톤을 맑게 개선해 주는 비타C엔자임세럼을 ...",활동적인 라이프스타일 속에서도 밝고 생기 있는 피부를 원한다면? 피부 톤 개선과 촉...,활동적인 라이프스타일 속에서도 맑고 생기있는 피부를 원하시나요? 비타C엔자임세럼은 ...,활동적인 라이프스타일에도 건강한 피부를 원하시나요? 피부 톤 개선과 수분 공급을 한...
1,1-0,"[131172564, 131173041]","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...",활기찬 하루를 보내면서도 피부에 자연스러운 광채와 탄력을 유지하고자 하는 열정적이고...,레티놀PDRN앰플,90%,"이 고객군은 피부의 광채와 탄력 유지에 대한 목표가 확고하고, 종합적인 안티에이징...",온종일 매끄럽고 탄력 있는 피부를 원하시나요? 레티놀PDRN앰플로 톤업과 주름 관리...,온종일 매끄럽고 탄력 있는 피부를 유지하고 싶으신가요? 강력한 피부 개선 효과를 선...,"온종일 매끄럽고 탄력 있는 피부를 원하시나요? 피부 톤 개선과 보습, 탄력 향상을 ...",온종일 매끄럽고 탄력 있는 피부를 원하시나요? 톤 개선과 촉촉한 보습을 제공하는 레...,하루 종일 매끈하고 탄력 있는 피부를 원하시나요? 피부 톤 개선과 탄력 향상에 탁월...
2,10-0,"[131174965, 131172564]","지구 환경을 생각하면서도 자신의 피부 건강과 외모를 꼼꼼히 관리하는 고객. 0번, ...",피부의 탄력을 높이고 젊고 생기 넘치는 모습을 유지하고 싶어하는 고객. 다양한 활동...,레티놀PDRN앰플,90%,이 고객군은 피부의 탄력과 생기를 높이고 자극을 최소화하면서도 전반적인 피부 개선...,피부 영양과 탄력을 유지하고 싶으시다면 레티놀 PDRN 앰플을 추천드립니다. 스트레...,스트레스와 외부 환경 요인으로부터 피부를 지키고 싶으신가요? 레티놀PDRN앰플로 탄...,스트레스로 인한 피부 순환 문제 걱정되시나요? 레티놀PDRN앰플이 피부 톤을 개선하...,피부 탄력과 영양이 걱정이라면? 레티놀PDRN앰플로 건강하고 생기있는 피부 케어를 ...,피부 순환과 탄력 걱정되시나요? 단단한 피부 탄력과 복합적인 문제를 해결해주는 레티...
3,10-2,"[111770997, 111506404]","자연 원료와 순면 제품을 선호하며, 피부와 자신을 소중하게 관리하는 고객. 0번, ...","자신의 피부를 부드럽게 감싸고, 자연의 순한 손길로 다독이는 것을 좋아하는 고객. ...",그린티씨드크림,90%,이 고객군은 피부의 민감성을 줄이고 자연의 순한 손길로 피부를 다독이는 것을 좋아...,새로운 여행과 모험을 떠날 준비가 되셨나요? 다양한 기획 세트와 리필용을 함께 소개...,"새로운 경험과 모험을 즐기는 당신에게 꼭 맞는 스킨케어 아이템, 그린티씨드세럼을 추...",새로운 모험을 즐기며 내 피부도 소중하게 챙기고 싶다면? 민감한 피부에도 자극 없이...,새로운 경험을 찾으세요? 여행 중에도 자극 없이 피부를 촉촉하게 유지하는 그린티씨드...,새로운 모험 속에서도 민감한 피부를 자극 없이 관리하고 싶으신가요? 피부를 진정시키...
4,11-0,"[131172879, 131174282]",자연 친화적인 성분과 제품을 선호하며 자신만의 고유한 향기와 스타일을 중요시하는 고...,섬세하고 감각적인 아름다움을 강조하는 고객. 주기적인 스트레스와 외부 환경으로 인한...,그린티씨드크림,85%,이 고객군은 주기적인 스트레스와 외부 환경으로 인해 건조함과 민감성을 겪고 있습니...,자신의 매력을 더욱 빛나게 하고 싶으신가요? 순한 성분으로 피부를 진정시키고 톤을 ...,고유의 매력을 더욱 돋보이고 싶으신가요? 예민한 피부에도 순하게 사용 가능한 레티놀...,화이트닝 관리가 필요하신가요? 부드러운 발림성과 민감한 피부에도 사용 가능한 레티놀...,피부가 민감하지만 세련된 매력을 한층 더하고 싶으신가요? 순하고 효과적인 레티놀시카...,"세련된 취향과 고유의 매력을 표현하고 싶다면, 순하면서도 강력한 레티놀 시카 앰플을..."
5,11-1,"[111770997, 111506404, 111506405]","이 고객은 다양한 피부 고민을 해결하고 자주 화장솜 등으로 피부를 관리하며, 철저한...","자주 돌아다니면서 새로운 환경에 노출되며, 다양한 장소에서 활동하는 스타일로, 지속...",레티놀PDRN앰플,85%,"이 고객군은 여러 가지 피부 고민을 효과적으로 해결하고자 하며, 다양한 상황에서 ...","끊임없이 새로운 곳을 탐험하는 당신, 피부도 지치지 않아야죠. 촉촉한 보습과 저자극...",끊임없이 새로운 곳을 탐험하시나요? 그린티씨드세럼은 빠르게 흡수돼 여행지에서도 피부...,"끊임없이 새로운 곳을 탐험하면서도 꼼꼼한 자기관리를 실천하는 당신께, 그린티씨드세럼...","끊임없이 새로운 곳을 탐험하면서도 자기관리에 철저한 당신에게, 그린티씨드세럼을 추천...",새로운 곳을 탐험하면서도 꼼꼼한 자기관리를 실천하는 당신에게 그린티 씨드 세럼을 추...
6,12-0,"[131174283, 131171020, 131173425]","피부를 건강하고 자연스럽게 유지하려는 신념을 가진 고객으로, 피부의 생기와 촉촉함을...","오랜 시간에도 생기를 잃지 않는 피부를 원하고, 항상 촉촉하게 유지되는 피부를 추구...",그린티씨드세럼,80%,이 고객군은 피부의 촉촉함을 항상 유지하고 싶어하며 자연스러운 상태를 중시합니다....,건강한 피부생기를 유지하고 자외선 차단의 중요성을 잘 알고 계시나요? 속건조를 잡아...,피부의 생기를 되찾고 싶으신가요? 자칫 자극받기 쉬운 피부에도 촉촉함을 선사하는 그...,당신의 피부에 건강한 생기를 더하고 싶으신가요? 자극 없이 촉촉함을 유지시켜주는 그...,피부에 생기를 더하고 촉촉함을 유지하고 싶으신가요? 천연 성분의 그린티씨드세럼이 일...,"일상에서 자외선도, 피부 자극도 잊고 싶으신가요? 끈적임 없이 촉촉하게 흡수되어 피..."
7,14-0,"[131172563, 131170641, 131175032]",자신만의 피부 관리와 세밀한 메이크업 루틴을 통해 완벽한 외모를 유지하려는 디테일한...,끊임없이 변화하는 환경 속에서도 완벽한 외모를 유지하고 싶은 열정을 지닌 고객. 피...,레티놀PDRN앰플,90%,"이 고객군은 끊임없이 변화하는 환경 속에서도 완벽한 외모를 유지하고자 하며, 다재...","외부 활동이 많아 피부가 자극받기 쉽다면, 레티놀 PDRN 앰플을 사용해보세요. 톤...","외부 환경에도 굴하지 않는 피부를 원하시나요? 탄력과 톤 개선, 보습까지 도와주는 ...","활동적인 라이프스타일로 언제나 완벽한 피부를 원하신다면, 레티놀PDRN앰플을 추천합...","활동적인 라이프스타일 속에서도 완벽한 피부 관리, 레티놀PDRN앰플과 함께하세요. ...","외부 활동에도 언제나 완벽한 피부를 유지하고 싶으신가요? 탄력 향상, 피부 톤 개선..."
8,15-2,"[131171021, 131173502, 131172564]","자연스럽고 건강한 피부 관리를 중요하게 생각하며, 자외선 차단과 노화 방지를 위해 ...",끊임없이 활동하며 매일의 다양하고 바쁜 일상 속에서도 피곤한 티 없이 건강하고 자연...,레티놀PDRN앰플,90%,이 고객군은 바쁜 일상 속에서도 피부의 전반적인 건강과 노화를 방지하려는

In [24]:
# 내부 + 외부 데이터 타겟팅

external_result = []
for i in tqdm(external_df["concern"]):
    
    system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
    user1 = f"""
    나는 너에게 특정 고객군의 피부 고민을 알려줄거야.
    그 특징을 바탕으로, [그린티씨드세럼, 그린티씨드크림, 레티놀시카앰플, 레티놀PDRN앰플] 중에서 가장 적절한 제품 하나를 추천해줘.
    그리고 그 추천이 얼마나 자신있는지 0% - 100%로 표시해준 후, 추천을 한 이유도 함께 설명해줘.
    우리가 선호하는 답변의 포맷은 다음과 같아: '레티놀시카앰플; 75%; 이 고객군은 피부가 예민하기 때문에, 순한 제품이 어울릴 것으로 예상됩니다.'
    각 제품에 대한 설명은 'Items'를 참조해.
    Items: {cp}
    """
    assistant1 = "알겠습니다. 고객군의 특징을 제공해 주세요."

    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": i}]}

    decision = True
    while decision:
        try:
            external_result.append(respond(payload))
            decision = False
        except:
            pass

one = []
two = []
three = []
for i in external_result:
    temp = i.split(";")
    one.append(temp[0])
    two.append(temp[1])
    three.append(temp[2])

external_df["recommended_item"] = one
external_df["confidence"] = two
external_df["reason_of_recommendation"] = three
external_df

100%|██████████| 97/97 [04:40<00:00,  2.89s/it]


,Unnamed: 0,internal_cluster_num,external_cluster_num,external_item,characteristics,internal_characteristics,concern,recommended_item,confidence,reason_of_recommendation,message0,message1,message2,message3,message4
0,0,0-0,0.0,"[PER_OTT 상, PER_MDLV 상]","시간과 에너지가 부족해 편리한 배달 음식을 주로 소비하지만, OTT 서비스를 통해 ...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...",바쁜 일상 속에서도 자신의 아름다움을 놓치지 않으려는 고객이에요. 지속되는 높은 생...,레티놀시카앰플,85%,이 고객군은 활발한 생활 리듬과 다양한 역할을 소화하면서 트러블과 모공 관리에 특...,바쁜 여행 일정 속에서도 피부 관리는 포기할 수 없죠? 빠르게 흡수되는 그린티씨드세...,여행 중에도 피부 건강을 놓칠 수 없죠? 빠른 흡수와 촉촉한 보습력을 자랑하는 그린...,빈틈없는 피부 관리가 필요하다면? 다양한 환경에서도 빠르고 편리하게 사용할 수 있는...,"여행 다니느라 바쁜 당신, 피부 관리 소홀히 하지 마세요. 촉촉하게 속건조를 잡아주...","여행 중에도 빼놓을 수 없는 피부 관리, 그린티씨드세럼으로 간편하게 해결해 보세요...."
1,1,0-0,1.0,"[PER_FRN_TRV 상, PER_DOM_TRV 상, PER_PRC_SNSV 상]","해외 여행에도 많은 관심을 가지고 있지만, 가격에 민감하여 합리적인 가격의 국내 여...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...","여러 새로운 경험을 추구하고, 활동적인 라이프스타일을 즐기는 이 고객은 피부에 다양...",레티놀PDRN앰플,80%,"이 고객군은 활동적인 라이프스타일로 인해 모공과 피부결 관리가 중요하며, 자주 건...",여행과 야외 활동 중에도 피부를 건강하게! 피부 톤을 밝히고 트러블을 완화해주는 레...,"여행 중 다양한 환경에 노출되는 피부, 걱정되시죠? 레티놀시카앰플은 진정과 보습, ...",다양한 환경에서도 피부 관리가 필요하시죠? 자극 없이 피부 톤을 환하게 잡아주는 레...,활동적인 라이프를 즐기면서도 비용도 중요하신가요? 피부 톤 개선과 트러블 완화에 효...,여행 속에서도 피부 트러블과 자극이 걱정되시나요? 보습과 트러블 완화는 물론 피부 ...
2,2,0-0,2.0,"[JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]","금전적 여유는 많지 않지만, 바쁜 회사 생활과 혼자만의 시간을 보내는 데 집중하는 ...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...",이 고객은 바쁜 일상 속에서도 자기만의 시간을 소중히 여기는 사람입니다. 금전적 여...,레티놀시카앰플,80%,이 고객군은 바쁜 일상 속에서도 자기관리를 중요시하며 특히 스트레스로 인한 피부 ...,바쁜 일상 속에서도 피부 관리가 필요하다면 레티놀 시카 앰플을 추천해 드립니다. 예...,바쁜 일상 속에서도 자기관리의 시간을 놓치고 싶지 않으신가요? 트러블 완화와 피부 ...,"바쁜 도심 속에서도 건강한 피부를 유지하고 싶다면, 레티놀시카앰플을 만나보세요. 이...",바쁜 일상 속에서도 트러블 관리가 필요하신가요? 짧은 시간에 피부 톤 개선과 보습을...,바쁜 도심 속에서도 피부 관리 포기하지 마세요. 트러블 완화와 피부 톤 개선에 효과...
3,3,1-0,0.0,"[PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상]","가격에 민감하지만, 편리함을 추구하며 배달 식품을 자주 먹으며 OTT 서비스를 즐기...","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...",활동성이 높으면서도 세밀한 자기 관리를 중요하게 여기는 이 고객은 라이프스타일 속 ...,레티놀PDRN앰플,90%,"이 고객군은 활동성이 높고 다양한 외부 요인에 피부가 쉽게 영향을 받기 때문에, ...",변화무쌍한 일상 속에서도 피부의 건강과 젊음을 유지하고 싶으세요? 피부 톤 개선과 ...,끊임없이 변화하는 일상 속에서도 탄력 있는 피부를 원하시나요? 주름 개선과 피부 톤...,"변화 많은 하루 중에도 피부의 건강과 젊음을 유지하고 싶다면, 레티놀PDRN앰플을 ...",매일 반복되는 스트레스 속에서도 피부의 젊음을 유지하고 싶으신가요? 주름 개선과 피...,끊임없이 변화하는 라이프스타일 속에서도 피부의 건강과 젊음을 유지하고 싶으신가요? ...
4,4,1-0,1.0,"[PER_DOM_TRV 상, PER_PRC_SNSV 상]","가성비를 중요하게 생각하며 주로 국내 여행에 관심을 두고 있는 고객. 1번, 2번 정보.","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...","활동적이고 잦은 이동을 하는 라이프스타일을 가진 당신, 여행 중에도 간편하게 들고 ...",레티놀PDRN앰플,90%,이 고객군은 활동적이고 이동이 잦아 간편하면서 고효율적인 스킨케어 제품을 찾고 있...,활발한 일상에도 깨끗한 피부 관리가 필요하다면? 주름 개선과 피부 탄력 향상에 탁월...,"활발한 일상을 보내는 당신의 피부, 건강하게 관리하고 싶으신가요? 강력한 안티에이징...",활발한 일상 속에서도 피부 관리를 놓치고 싶지 않으신가요? 피부 탄력과 주름 개선을...,"어디서든 피부를 신경 쓰시는 당신께, 레티놀 PDRN 앰플을 추천합니다. 주름 개선...","언제 어디서든 피부 건강을 빠르게 챙기고 싶다면, 레티놀 PDRN 앰플을 추천드려요..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,7-1,4.0,"[PER_FRN_TRV 상, PER_DOM_TRV 상]",다양한 문화 체험과 여가 활동을 즐기며 국내외 여행에 많은 관심을 갖고 있는 고객....,자주 트러블이 발생하는 피부 고민을 다양한 제품을 통해 체계적으로 관리하고 있는 고...,"다채로운 생활 속에서 다양한 경험을 중시하고 있으며, 번잡한 일상에서 벗어나 자연스...",레티놀시카앰플,80%,이 고객군은 번잡한 일상 속에서 피부 스트레스를 줄이고 트러블이 발생할 경우 즉각...,"끊임없이 새로운 도전에 나서는 당신, 여행 중 예민해진 피부도 빠르게 진정시킬 수 ...","늘 새로운 도전을 즐기며 열정적인 삶을 살아가시는 당신께, 피부를 지키는 완벽한 동...","늘 새로운 도전을 즐기는 당신, 레티놀시카앰플로 피부 관리도 업그레이드하세요. 피부...","끊임없이 도전하며 세상을 탐험하는 당신, 민감해진 피부도 놓칠 수 없죠. 트러블 완...",늘 도전과 열정으로 가득 찬 당신에게 필요한 레티놀시카앰플을 추천드립니다. 다양한 ...
93,93,7-1,5.0,"[LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상]",바쁜 직장 생활과 자녀 돌봄의 이중 부담 속에서 편리함을 중시하여 배달 식품을 자주...,자주 트러블이 발생하는 피부 고민을 다양한 제품을 통해 체계적으로 관리하고 있는 고...,강한 책임감과 열정으로 매일을 살아가는 고객. 하지만 바쁜 일상 속에서 피부 관리에...,레티놀시카앰플,85%,"이 고객군은 바쁜 일상 속에서도 실용적이고 효과적인 스킨케어 솔루션을 원하며, 피...",피부 트러블에 지치셨나요? 예민한 피부에도 순하게 작용하는 레티놀시카앰플을 만나보세...,"피부 트러블로 고민이신가요? 트러블 완화와 보습을 동시에, 레티놀 시카 앰플로 매끈...",피부 트러블로 고민 중이신가요? 레티놀 시카 앰플로 트러블을 완화하고 매끈한 피부를...,"자주 발생하는 피부 트러블로 고민이시라면, 레티놀시카앰플을 추천해드립니다. 트러블 ...",피부 트러블 관리 고민 중이신가요? 순한 성분으로 트러블을 완화시키며 보습과 피부 ...
94,94,8-0,2.0,"[JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]",높은 생활비

In [28]:
internal_df.to_csv("internal_characteristics.csv", encoding = "utf-8-sig")
external_df.to_csv("external_characteristics.csv", encoding = "utf-8-sig")

In [26]:
print(internal_df["recommended_item"].unique())
print(external_df["recommended_item"].unique())

['레티놀PDRN앰플' '그린티씨드크림' '그린티씨드세럼' '레티놀시카앰플']
['레티놀시카앰플' '레티놀PDRN앰플' '1. **활동적인 라이프스타일과 피부 고민에 대한 특징:**\n    레티놀시카앰플'
 '그린티씨드세럼' '그린티씨드크림']
